In [8]:

%load_ext autoreload
%autoreload 2
import sys
# map folder using sys
#sys.path.append('../')
import pandas as pd
import numpy as np
from galgoz import DATA_FOLDER
from galgoz import Galgoz
import numpy as np
from plotly import graph_objects as go
from talib import RSI as rsi
from galgoz.indicators.oscillators import qqe


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
filename = DATA_FOLDER / "GBP_JPY_H1.pkl"
data = pd.read_pickle(filename)
data["time"] = pd.to_datetime(data["time"])
data.set_index("time", inplace=True)
data["time_str"] = data.index.strftime(" %-b %d, '%y %H:%M")
data[["mid_o", "mid_h", "mid_l", "mid_c"]] = data[["mid_o", "mid_h", "mid_l", "mid_c"]].astype(float)

In [17]:
def qqe_nt(data: pd.DataFrame, length: int = 8, smooth: int = 1, factor: float = 1.618) -> pd.DataFrame:
    df = data.copy()
    qqe_fast = rsi(df["mid_c"], timeperiod=length).ewm(span=smooth).mean().values
    rsi_delta = qqe_fast.diff().abs().values
    wmma = np.zeros(len(df))
    atr_rsi = np.zeros(len(df))
    for i in range(1, len(df)):
        wmma[i] = rsi_delta[i] / length + (1 - 1 / length) * (wmma[i - 1] if not np.isnan(wmma[i - 1]) else 0)
        atr_rsi[i] = wmma[i] / length + (1 - 1 / length) * (atr_rsi[i - 1] if not np.isnan(atr_rsi[i - 1]) else 0)
    qqe_up=df.qqe_fast.values+atr_rsi*factor
    qqe_down=df.qqe_fast.values-atr_rsi*factor

    qqe_slow=np.zeros(len(df))
    for i in range(1, len(df)):
        if qqe_up[i] < (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0):
            qqe_slow[i]=qqe_up[i]
        elif qqe_fast[i] > (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0) and qqe_fast[i-1] < (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0):
            qqe_slow[i]=qqe_down[i]
        elif qqe_down[i] > (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0):
            qqe_slow[i]=qqe_down[i]
        elif qqe_fast[i] < (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0) and qqe_fast[i-1] > (qqe_slow[i-1] if not np.isnan(qqe_slow[i-1]) else 0):
            qqe_slow[i]=qqe_up[i]
        else:
            qqe_slow[i]=qqe_slow[i-1]
    
    df["qqe_fast"]=qqe_fast
    df["qqe_slow"]=qqe_slow
    return df

#df = qqe_nt(data)
df2 = qqe(data)

df2.tail(10)




,complete,volume,mid_o,mid_h,mid_l,mid_c,time_str,qqe_fast,qqe_slow
time,,,,,,,,,
2024-12-27 12:00:00+00:00,1,11214.0,197.886,198.146,197.831,198.096,"Dec 27, '24 12:00",67.658376,56.264235
2024-12-27 13:00:00+00:00,1,12424.0,198.092,198.407,198.046,198.405,"Dec 27, '24 13:00",74.588375,63.170982
2024-12-27 14:00:00+00:00,1,15587.0,198.409,198.508,198.096,198.374,"Dec 27, '24 14:00",72.799842,63.170982
2024-12-27 15:00:00+00:00,1,21613.0,198.382,198.509,198.050,198.172,"Dec 27, '24 15:00",61.769662,73.085602
2024-12-27 16:00:00+00:00,1,17683.0,198.168,198.406,198.070,198.270,"Dec 27, '24 16:00",64.732409,73.085602
2024-12-27 17:00:00+00:00,1,14708.0,198.268,198.274,198.018,198.234,"Dec 27, '24 17:00",62.692684,73.085602
2024-12-27 18:00:00+00:00,1,12225.0,198.234,198.392,198.197,198.365,"Dec 27, '24 18:00",67.015093,73.085602
2024-12-27 19:00:00+00:00,1,11845.0,198.360,198.714,198.314,198.714,"Dec 27, '24 19:00",75.616567,64.910356
2024-12-27 20:00:00+00:00,1,9335.0,198.714,198.718,198.528,198.650,"Dec 27, '24 20:00",71.698138,64.910356


In [11]:
df2.tail(10)

,complete,volume,mid_o,mid_h,mid_l,mid_c,time_str,qqe_fast,qqe_slow
time,,,,,,,,,
2024-12-27 12:00:00+00:00,1,11214.0,197.886,198.146,197.831,198.096,"Dec 27, '24 12:00",67.658376,56.264235
2024-12-27 13:00:00+00:00,1,12424.0,198.092,198.407,198.046,198.405,"Dec 27, '24 13:00",74.588375,63.170982
2024-12-27 14:00:00+00:00,1,15587.0,198.409,198.508,198.096,198.374,"Dec 27, '24 14:00",72.799842,63.170982
2024-12-27 15:00:00+00:00,1,21613.0,198.382,198.509,198.050,198.172,"Dec 27, '24 15:00",61.769662,73.085602
2024-12-27 16:00:00+00:00,1,17683.0,198.168,198.406,198.070,198.270,"Dec 27, '24 16:00",64.732409,73.085602
2024-12-27 17:00:00+00:00,1,14708.0,198.268,198.274,198.018,198.234,"Dec 27, '24 17:00",62.692684,73.085602
2024-12-27 18:00:00+00:00,1,12225.0,198.234,198.392,198.197,198.365,"Dec 27, '24 18:00",67.015093,73.085602
2024-12-27 19:00:00+00:00,1,11845.0,198.360,198.714,198.314,198.714,"Dec 27, '24 19:00",75.616567,64.910356
2024-12-27 20:00:00+00:00,1,9335.0,198.714,198.718,198.528,198.650,"Dec 27, '24 20:00",71.698138,64.910356


In [12]:
from plotly.subplots import make_subplots

# Use only the last 1000 data points
df_last_1000 = df.tail(1000)

# Create subplots with 2 rows and 1 column, with specified row heights
fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], shared_xaxes=True)

# Add the candlestick plot to the first row
fig.add_trace(
    go.Candlestick(
        x=df_last_1000['time_str'],
        open=df_last_1000['mid_o'],
        high=df_last_1000['mid_h'],
        low=df_last_1000['mid_l'],
        close=df_last_1000['mid_c'],
        name='Candlesticks'
    ),
    row=1, col=1
)

# Add the qqe_fast and qqe_slow plots to the second row
fig.add_trace(
    go.Scatter(x=df_last_1000['time_str'], y=df_last_1000['qqe_fast'], name='QQE Fast'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df_last_1000['time_str'], y=df_last_1000['qqe_slow'], name='QQE Slow'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=800, title_text="Candlestick and QQE Indicators", xaxis_rangeslider_visible=False)

# Show the plot
fig.show()

NameError: name 'df' is not defined

In [ ]:
wmma

In [ ]:
from plotly.subplots import make_subplots

# Create subplots with 2 rows and 1 column, with specified row heights
fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3])

# Add the close price plot to the first row
fig.add_trace(
    go.Scatter(x=data.time_str, y=data['mid_c'], name='Close Price'),
    row=1, col=1
)

# Add other plots to the second row (example: qqe_fast and qqe_slow)
fig.add_trace(
    go.Scatter(x=df.index, y=df['qqe_fast'], name='QQE Fast'),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=df.index, y=df['qqe_slow'], name='QQE Slow'),
    row=2, col=1
)

# Update layout
fig.update_layout(height=800, title_text="Close Price and QQE Indicators")

# Show the plot
fig.show()

In [22]:
df = qqe(data)

In [ ]:
df.tail()